# Loading Pre-extracted Vector Embeddings into Pandas DataFrames
This notebook demonstrates how to load and work with pre-extracted vector embeddings into pandas DataFrames.<br>
Vector embeddings are numerical representations of images that capture semantic meaning in high-dimensional space.

In [1]:
# Core data manipulation and numerical computing
import os
import numpy as np
import pandas as pd

# Progress tracking for long-running operations
from tqdm import tqdm

# Type hints for better code documentation and IDE support
from typing import List, Literal, Tuple, Optional

# Concurrent processing for improved performance
from concurrent.futures import ThreadPoolExecutor, as_completed

## Core Functions for Loading Embeddings
The following functions handle the loading of pre-extracted vector embeddings from disk into pandas DataFrames.<br>
The implementation uses parallel processing to efficiently load multiple embedding files simultaneously.

In [2]:
def _load_single_embedding(sop: str, dataset_path: str, FM: str) -> Tuple[str, Optional[np.ndarray]]:
    """
    Load a single embedding file from disk.
    
    This helper function handles the loading of individual .npy files containing 
    pre-computed vector embeddings. It includes error handling for missing files
    and corrupted data.
    
    Args:
        sop (str): SOP (Study/Series/Image) identifier - unique ID for the embedding
        dataset_path (str): Base path to the dataset embeddings directory
        FM (str): Feature model name (e.g., 'RAD-DINO', 'MedImageInsights')
    
    Returns:
        Tuple[str, Optional[np.ndarray]]: 
            - First element: the SOP identifier (for tracking)
            - Second element: loaded embedding array or None if loading failed
    """
    # Construct the full path to the embedding file
    # Format: dataset_path/embds_{FM}/{sop}.npy
    embedding_file = os.path.join(dataset_path, f'embds_{FM}', sop + '.npy')
    
    # Check if the embedding file exists before attempting to load
    if not os.path.exists(embedding_file):
        print(f"Warning: Embedding file not found for SOP '{sop}': {embedding_file}")
        return sop, None
        
    try:
        # Load the numpy array from disk
        # Embeddings are typically stored as .npy files for efficient loading
        embd = np.load(embedding_file)
        return sop, embd
    except Exception as e:
        # Handle any errors during file loading (corruption, permissions, etc.)
        print(f"Error loading embedding for SOP '{sop}': {e}")
        return sop, None

def load_embeddings(
    SOP: List[str], 
    dataset: Literal['EmoryCXR','MIMIC','MRKR','EMBED'] = 'EmoryCXR', 
    FM: Literal['MedImageInsights','RAD-DINO','CheXagent','MedGemma','Mammo-CLIP','BiomedCLIP'] = 'RAD-DINO',
    max_workers: Optional[int] = None
) -> pd.DataFrame:
    """
    Load embeddings for multiple SOPs in parallel and return as a pandas DataFrame.
    
    This function efficiently loads vector embeddings for a list of identifiers using
    parallel processing. The embeddings are loaded from pre-computed .npy files and
    organized into a pandas DataFrame for easy manipulation and analysis.
    
    Args:
        SOP (List[str]): List of SOP identifiers to load embeddings for
        dataset (Literal): Dataset name - specifies which dataset's embeddings to load
            Options: 'EmoryCXR', 'MIMIC', 'MRKR', 'EMBED'
        FM (Literal): Feature model name - specifies which embedding model to use
            Options: 'MedImageInsights', 'RAD-DINO', 'CheXagent', 'MedGemma', 'Mammo-CLIP'
        max_workers (Optional[int]): Maximum number of parallel worker threads
            Limited to 4 workers max to prevent system overload. Defaults to 4.
    
    Returns:
        pd.DataFrame: DataFrame with:
            - Rows: Each SOP identifier
            - Columns: Embedding dimensions (typically 768, 1024, or 2048 features)
            - Index: 'SOP' column containing the identifiers
    
    Raises:
        TypeError: If SOP is not a list
        ValueError: If SOP list is empty or invalid dataset/FM specified
        RuntimeError: If no embeddings could be loaded successfully
    """
    # Input validation - ensure SOP is a list
    if not isinstance(SOP, list):
        raise TypeError("SOP must be a list of identifiers")
    
    # Input validation - ensure SOP list is not empty
    if not SOP:
        raise ValueError("SOP list cannot be empty")
    
    # Dataset path mapping - defines where embeddings are stored for each dataset
    path_dict = {
        'EmoryCXR': '/mnt/NAS3/Embeddings/EmoryCXR/',
        'MIMIC': '/mnt/NAS3/Embeddings/MIMIC/',
        'MRKR': '/mnt/NAS3/Embeddings/MRKR/',
        'EMBED': '/mnt/NAS3/Embeddings/EMBED/'
    }
    
    # Validate dataset parameter
    if dataset not in path_dict:
        raise ValueError(f"Dataset '{dataset}' not supported. Available: {list(path_dict.keys())}")
    
    # Supported feature models - each produces different embedding dimensions
    FMs = ['MedImageInsights','RAD-DINO','CheXagent','MedGemma','Mammo-CLIP','BiomedCLIP']
    if FM not in FMs:
        raise ValueError(f"FM '{FM}' not supported. Available: {FMs}")
    
    # Get the base path for the selected dataset
    dataset_path = path_dict[dataset]
    
    # Dictionary to store successfully loaded embeddings
    embds_dict = {}
    
    # Parallel loading with thread pool
    # Limit to 4 workers maximum to prevent overwhelming the file system
    effective_max_workers = min(max_workers, 4) if max_workers is not None else 4
    
    with ThreadPoolExecutor(max_workers=effective_max_workers) as executor:
        # Submit all loading tasks to the thread pool
        future_to_sop = {
            executor.submit(_load_single_embedding, sop, dataset_path, FM): sop 
            for sop in SOP
        }
        
        # Collect results as they complete, with progress tracking
        for future in tqdm(as_completed(future_to_sop), 
                          total=len(SOP), 
                          desc=f"Loading {FM} embeddings"):
            sop, embedding = future.result()
            
            # Only store successfully loaded embeddings
            if embedding is not None:
                embds_dict[sop] = embedding
    
    # Ensure at least some embeddings were loaded successfully
    if not embds_dict:
        raise RuntimeError("No embeddings were successfully loaded")
    
    # Convert dictionary to DataFrame
    # - Keys become column names (SOPs)
    # - Values become column data (embedding vectors)
    # - Transpose so SOPs become rows instead of columns
    # - Reset index to make SOP identifiers a proper column
    df = pd.DataFrame(embds_dict).T.reset_index(names=['SOP'])
    
    print(f"Successfully loaded {len(embds_dict)} embeddings out of {len(SOP)} requested")
    print(f"Embedding shape: {df.shape[1]-1}")
    
    return df

## Metadata loading
**Available Embeddings:**<br>
<img src="./figs/embeddings.png" alt="login" width="750"><br>

In [3]:
# =============================================================================
# DATASET AND MODEL CONFIGURATION
# =============================================================================
# This section defines the core configuration for loading embeddings.
# Modify these variables to switch between datasets or feature models.

# Primary dataset selection
DATASET = 'EmoryCXR'  # Current dataset being analyzed
                      # Options: 'EmoryCXR', 'MIMIC', 'MRKR', 'EMBED'

# Feature model selection
FM = 'MedGemma'          # Embedding model to use for analysis
                         # Available models:
                         # - 'MedImageInsights'
                         # - 'RAD-DINO'
                         # - 'CheXagent'
                         # - 'MedGemma'
                         # - 'Mammo-CLIP'
                         # - 'BiomedCLIP'

# =============================================================================
# DATASET-SPECIFIC FILE PATHS
# =============================================================================
# These paths point to the metadata and embedding availability files.
# Each dataset has its own directory structure and naming conventions.

# Main metadata file containing clinical and imaging information
META_PATH = '/mnt/NAS3/CXR/EmoryCXRv2/TABLES/EmoryCXR_v2_Metadata_08112025.csv'
# Dataset alternatives (uncomment and modify DATASET variable above):
# MIMIC: '/mnt/NAS3/CXR/MIMIC_CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv'
# MRKR:  '/mnt/NAS3/MRKR_dataset/MRKR_processed_images_08112025.csv'
# EMBED: '/mnt/NAS3/EMBED_dataset/EMBED_metadata.csv'

# Embedding availability list - tracks which images have pre-computed embeddings
EMBDS_LIST_PATH = '/mnt/NAS3/Embeddings/EmoryCXR/embds_list.csv'
# This CSV contains boolean columns for each feature model indicating availability
# Dataset alternatives:
# MIMIC: '/mnt/NAS3/Embeddings/MIMIC/embds_list.csv'
# MRKR:  '/mnt/NAS3/Embeddings/MRKR/embds_list.csv'
# EMBED: '/mnt/NAS3/Embeddings/EMBED/embds_list.csv'

# =============================================================================
# DATASET-SPECIFIC IDENTIFIER COLUMNS
# =============================================================================
# Different datasets use different column names for image identifiers.
# This mapping ensures compatibility across datasets.

IMAGE_ID_COLUMN = 'SOP'  
# Dataset-specific identifier columns:
# EmoryCXR: 'SOP' 
# MRKR: 'SOP'
# MIMIC: 'dicom_id'  
# EMBED: 'SOP'  

# =============================================================================
# QUICK DATASET SWITCHING GUIDE
# =============================================================================
# To switch datasets, update these three variables:
# 
# For MIMIC-CXR:
#   DATASET = 'MIMIC'
#   IMAGE_ID_COLUMN = 'dicom_id'
#   Update META_PATH and EMBDS_LIST_PATH to MIMIC paths
#
# For MRKR:
#   DATASET = 'MRKR' 
#   IMAGE_ID_COLUMN = 'SOP'
#   Update META_PATH and EMBDS_LIST_PATH to MRKR paths
#
# For EMBED:
#   DATASET = 'EMBED'
#   IMAGE_ID_COLUMN = 'SOP'
#   Update META_PATH and EMBDS_LIST_PATH to EMBED paths

In [4]:
# Load the main metadata file
if DATASET == 'EMBED':
    meta = pd.read_parquet(META_PATH)
else:
    meta = pd.read_csv(META_PATH)

# Load the list of available embeddings
embds_list = pd.read_csv(EMBDS_LIST_PATH)

# Merge metadata with embedding availability
meta = meta.merge(embds_list, on=IMAGE_ID_COLUMN, how='inner')

# Display the merged metadata
meta

,empi_anon,AccessionNumber_anon,StudyDate_anon,StudyDescription,SeriesDescription,Sex,Age,BMI,ImagePath,Race,...,CLAHE,Frontal,DuplicateNumber,Outlier,SOP,CheXagent,MedImageInsights,RAD-DINO,MedGemma,BiomedCLIP
0,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.11539031758371223425...,True,True,True,True,True
1,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.75909224393525203125...,True,False,True,True,True
2,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,1.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.83012792870623702441...,True,False,True,True,True
3,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.10019530719819746433...,True,False,True,True,True
4,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,1.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.99536058454842266521...,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486497,51508665,778743244229,2021-09-29 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,Lateral,M,34.0,24.4,51508665/778743244229/1.2.826.0.1.3680043.8.49...,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.85700220699029757330...,True,False,True,True,True
2486498,11580015,217914768130,2021-08-02 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,PA,F,23.0,22.0,11580015/217914768130/1.2.826.0.1.3680043.8.49...,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.11672472902184660109...,True,True,True,True,True
2486499,11580015,217914768130,2021-08-02 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,LATERAL,F,23.0,22.0,11580015/217914768130/1.2.826.0.1.3680043.8.49...,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.88094550305034295909...,True,False,True,True,True
2486500,23754864,126932809055,2021-08-29 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,Chest,F,82.0,23.3,23754864/126932809055/1.2.826.0.1.3680043.8.49...,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.48191769673257009848...,True,True,True,True,True


### Availability checks

In [5]:
# Filter metadata to only include images with embeddings for the selected feature model
# The metadata contains boolean columns indicating embedding availability for each model
print(f"Selected feature model: {FM}")
print(f"Total images in metadata: {len(meta)}")

# Create subset containing only images with embeddings for the selected FM
# meta[FM] creates a boolean mask - True where embeddings exist for this model
meta_sub = meta[meta[FM]].reset_index(drop=True)

print(f"Images with {FM} embeddings: {len(meta_sub)}")

meta_sub

Selected feature model: MedGemma
Total images in metadata: 2486502
Images with MedGemma embeddings: 2486502


,empi_anon,AccessionNumber_anon,StudyDate_anon,StudyDescription,SeriesDescription,Sex,Age,BMI,ImagePath,Race,...,CLAHE,Frontal,DuplicateNumber,Outlier,SOP,CheXagent,MedImageInsights,RAD-DINO,MedGemma,BiomedCLIP
0,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.11539031758371223425...,True,True,True,True,True
1,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.75909224393525203125...,True,False,True,True,True
2,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,1.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.83012792870623702441...,True,False,True,True,True
3,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.10019530719819746433...,True,False,True,True,True
4,84269073,3739633430788944,2009-06-19 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,XR Chest 2 Views PA + Lateral,F,62.0,NaN,84269073/3739633430788944/1.2.826.0.1.3680043....,White,...,1.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.99536058454842266521...,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486497,51508665,778743244229,2021-09-29 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,Lateral,M,34.0,24.4,51508665/778743244229/1.2.826.0.1.3680043.8.49...,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.85700220699029757330...,True,False,True,True,True
2486498,11580015,217914768130,2021-08-02 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,PA,F,23.0,22.0,11580015/217914768130/1.2.826.0.1.3680043.8.49...,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.11672472902184660109...,True,True,True,True,True
2486499,11580015,217914768130,2021-08-02 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,LATERAL,F,23.0,22.0,11580015/217914768130/1.2.826.0.1.3680043.8.49...,White,...,0.0,0,0.0,0.0,1.2.826.0.1.3680043.8.498.88094550305034295909...,True,False,True,True,True
2486500,23754864,126932809055,2021-08-29 00:00:00,XR Chest 2 Views PA + Lat Stnd Protocol,Chest,F,82.0,23.3,23754864/126932809055/1.2.826.0.1.3680043.8.49...,White,...,0.0,1,0.0,0.0,1.2.826.0.1.3680043.8.498.48191769673257009848...,True,True,True,True,True


## Sample selection

Here is just an example. Select samples based on you own tasks.

In [6]:
# Select samples you want to analyze
# For large datasets, working with a subset can speed up development and testing

samples = meta_sub.sample(n=10000) # random select for example

# Extract the list of SOP identifiers from our sample metadata
sop_list = list(samples[IMAGE_ID_COLUMN])

## Embedding loading

In [7]:
# Load embeddings using parallel processing
# This will create a DataFrame with SOPs as rows and embedding dimensions as columns
df = load_embeddings(
    SOP=sop_list,           # List of identifiers to load embeddings for
    dataset=DATASET,     # Dataset name (matches our metadata)
    FM=FM                   # Feature model (matches our filtered selection)
)

Loading MedGemma embeddings: 100%|██████████| 10000/10000 [00:07<00:00, 1332.47it/s]


Successfully loaded 10000 embeddings out of 10000 requested
Embedding shape: 1152


In [ ]:
df

## Example Application: Logistic Regression with Embeddings
This section demonstrates how to use the loaded vector embeddings for supervised learning tasks. We'll use logistic regression to classify medical findings based on the embedding features.

In [9]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [10]:
# Load the finding labels for supervised learning
# This file contains binary labels for various medical findings/diagnoses
print("Loading finding labels...")
label = pd.read_csv('/mnt/NAS3/CXR/EmoryCXRv2/TABLES/EmoryCXR_v2_FindingLabel_10162024.csv')

# Step 1: Fill NaN values with 0.0 (assuming not mentioned = negative/absent)
label = label.fillna(0.0)
print(f"After filling NaN with 0.0: {label.shape}")

# Step 2: Handle uncertain/unknown labels (coded as -1)
# Replace -1 with NaN to mark uncertain cases, then remove them
label_before_cleaning = len(label)
label = label.replace({-1: np.nan}).dropna()
print(f"After removing uncertain labels (-1): {label.shape}")
print(f"Removed {label_before_cleaning - len(label)} uncertain cases")

# Show the cleaned label data
label

Loading finding labels...
After filling NaN with 0.0: (1323308, 15)
After removing uncertain labels (-1): (1061624, 15)
Removed 261684 uncertain cases


,AccessionNumber_anon,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,3739633430788944,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3002096846959729,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3881172571748277,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1335965105872601,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7913541839793330,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323303,403507296803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1323304,336608658041,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1323305,301954054836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1323306,839318589171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Linking Embeddings to Study-Level Identifiers
To connect our embeddings with the label data, we need to create a mapping between image-level identifiers (SOPs) and study-level identifiers (Accession Numbers). This is essential because labels are often assigned at the study level while embeddings exist at the image level.

In [11]:
# Create mapping between SOP (image-level) and AccessionNumber (study-level)

SOP_ACC_mapping = meta[['AccessionNumber_anon','SOP']].drop_duplicates()
df = df.merge(SOP_ACC_mapping, on='SOP')
df

,SOP,0,1,2,3,4,5,6,7,8,...,1143,1144,1145,1146,1147,1148,1149,1150,1151,AccessionNumber_anon
0,1.2.826.0.1.3680043.8.498.90442845168613638599...,-0.039621,0.014772,0.011734,0.021557,0.054431,0.037025,-0.011072,-0.028565,-0.028873,...,-0.051175,-0.054008,0.000549,-0.013997,0.031702,0.001969,0.001439,0.025983,-0.025922,722767873890
1,1.2.826.0.1.3680043.8.498.11690223087254205844...,-0.021981,0.029411,0.011351,0.013693,0.016749,0.039901,-0.006683,-0.041209,0.005029,...,-0.050264,-0.074120,0.035675,0.004343,0.007692,0.022390,0.022333,-0.000020,-0.024174,782788494151
2,1.2.826.0.1.3680043.8.498.67183300301457083053...,-0.039476,0.013035,0.006208,0.007680,-0.028006,0.031921,-0.031406,-0.024710,0.002265,...,0.041645,-0.010188,0.043828,0.052161,-0.021916,0.032130,-0.010035,0.014297,-0.005059,5263849321298054
3,1.2.826.0.1.3680043.8.498.84509792207375040961...,-0.055685,0.011214,0.009754,-0.005672,0.037304,0.068742,-0.046513,-0.044090,-0.001188,...,-0.053900,-0.035551,-0.007376,0.002554,-0.006675,-0.004452,-0.040846,0.006240,-0.021441,1873042729813467
4,1.2.826.0.1.3680043.8.498.69494023959192938140...,-0.062988,0.016038,-0.013200,-0.008242,0.009557,-0.005827,-0.009251,-0.047648,0.017968,...,-0.025179,-0.033228,0.029376,0.011582,0.022188,0.017616,-0.010140,-0.021940,-0.033488,7964502561462963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.2.826.0.1.3680043.8.498.29247806434232235963...,-0.011814,0.012107,-0.000038,0.003010,-0.007067,0.023315,-0.021251,-0.043308,0.025618,...,-0.044101,-0.101472,0.000093,-0.009582,0.017405,0.014640,-0.002184,0.040105,-0.052369,977018412778
9996,1.2.826.0.1.3680043.8.498.66889556334917604725...,-0.008753,0.012398,0.003556,0.023984,0.010179,0.020502,-0.054401,0.003335,0.000407,...,-0.077779,-0.053279,0.017376,0.019912,0.016962,-0.013758,0.019010,0.040687,-0.011727,5049575610003907
9997,1.2.826.0.1.3680043.8.498.40024303053958378872...,-0.037328,0.025761,-0.011934,0.007965,0.003649,0.009692,-0.008007,-0.035816,0.039070,...,-0.013266,-0.011561,0.001135,-0.003803,-0.010804,-0.000613,-0.040669,-0.015998,-0.040787,308827029640
9998,1.2.826.0.1.3680043.8.498.96065924676345628131...,-0.036217,0.065120,-0.018003,0.018365,-0.005198,0.033650,0.012903,-0.034124,0.000765,...,-0.010350,-0.030367,0.018763,0.038585,0.007880,0.006794,-0.035801,-0.006873,-0.001445,7954314245514328


### Merging Embeddings with Labels
Now we'll combine our embedding features with the ground truth labels to create the final dataset for machine learning. This step links the vector representations with the clinical findings.

In [12]:
df = df.merge(label, on='AccessionNumber_anon')
df

,SOP,0,1,2,3,4,5,6,7,8,...,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,1.2.826.0.1.3680043.8.498.67183300301457083053...,-0.039476,0.013035,0.006208,0.007680,-0.028006,0.031921,-0.031406,-0.024710,0.002265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2.826.0.1.3680043.8.498.84509792207375040961...,-0.055685,0.011214,0.009754,-0.005672,0.037304,0.068742,-0.046513,-0.044090,-0.001188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.2.826.0.1.3680043.8.498.69494023959192938140...,-0.062988,0.016038,-0.013200,-0.008242,0.009557,-0.005827,-0.009251,-0.047648,0.017968,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.2.826.0.1.3680043.8.498.10397104015287622368...,-0.049240,0.040114,-0.017875,-0.006010,-0.002483,0.001574,0.007444,-0.009471,0.001015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.2.826.0.1.3680043.8.498.17873126746280110047...,-0.038912,0.024210,0.023269,0.011660,0.013934,0.067171,-0.019209,-0.026333,0.005833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8012,1.2.826.0.1.3680043.8.498.22469397826102206182...,-0.017924,0.025457,-0.015692,0.012558,-0.005567,0.048665,-0.056532,-0.023608,0.022681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8013,1.2.826.0.1.3680043.8.498.10446169478185566298...,-0.063617,0.005107,0.017118,0.000017,0.030078,0.029108,0.001096,-0.027731,-0.015919,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8014,1.2.826.0.1.3680043.8.498.40024303053958378872...,-0.037328,0.025761,-0.011934,0.007965,0.003649,0.009692,-0.008007,-0.035816,0.039070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8015,1.2.826.0.1.3680043.8.498.96065924676345628131...,-0.036217,0.065120,-0.018003,0.018365,-0.005198,0.033650,0.012903,-0.034124,0.000765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Logistic Regression Classification Example
Now we'll demonstrate how to use the loaded embeddings for a supervised learning task. We'll build a logistic regression classifier to predict cardiomegaly (enlarged heart) from the embedding features.

In [16]:
X = df[[i for i in range(1024)]] # ex: 1024 for MedGemma
y = df['Cardiomegaly']

# Proper train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Train on training data only
clf = LogisticRegression(
    random_state=0,
    max_iter=1000,
    solver='lbfgs',
    class_weight='balanced'  # Handle imbalanced classes
).fit(X_train, y_train)

# Evaluate on both sets for comparison
train_accuracy = clf.score(X_train, y_train)
test_accuracy = clf.score(X_test, y_test)

print('train accuracy:', train_accuracy)
print('test accuracy:', test_accuracy)

train accuracy: 0.7888663651956962
test accuracy: 0.7599750623441397
